In [ ]:
for(cnt in 1:7){
    boundary = data.frame() #sensor_name , boundary_min , boundary_max , outlier_number
    print(boundary)
    #============================================================
    folder = paste("./RBG", cnt, sep="")
    my_files <- list.files(folder)

    my_files2 = paste(folder, "/",my_files, sep="")

    all_csv <- lapply(my_files2,read.csv)

    names(all_csv) <- gsub(".csv","",
                           list.files(folder,full.names = FALSE),
                           fixed = TRUE)

    #============================================================

    for (i in 1:length(all_csv)){

        RBG1_drive_gear_a_max = as.data.frame(all_csv[i])
        names(RBG1_drive_gear_a_max) = c("end_time","ifm_analysis_sensordata_for_datathon_final.realvalue")

            sep_time_RBG1_drive_gear_a_max = separate(RBG1_drive_gear_a_max, end_time, c('end_time_date','end_time_time'), sep = " ")
        head(sep_time_RBG1_drive_gear_a_max)

        # day
        Agg_day_RBG1_drive_gear_a_max= summarise(group_by(sep_time_RBG1_drive_gear_a_max, end_time_date), mean_realvalue=mean(ifm_analysis_sensordata_for_datathon_final.realvalue,na.rm = TRUE))
        head(Agg_day_RBG1_drive_gear_a_max)

        # g = ggplot(data = Agg_day_RBG1_drive_gear_a_max, aes(x = mean_realvalue, y = end_time_date ))
        # g = g + geom_point()
        # print(g)

        # separate time
        sep_time_RBG1_drive_gear_a_max['end_time_hour']=substr(sep_time_RBG1_drive_gear_a_max$end_time_time, 1, 2)
        sep_time_RBG1_drive_gear_a_max['end_time_min']=substr(sep_time_RBG1_drive_gear_a_max$end_time_time, 4, 5)
        sep_time_RBG1_drive_gear_a_max['end_time_sec']=substr(sep_time_RBG1_drive_gear_a_max$end_time_time, 7, 8)
        head(sep_time_RBG1_drive_gear_a_max)

        # hour
        Agg_day_hour_RBG1_drive_gear_a_max= summarise(group_by(sep_time_RBG1_drive_gear_a_max, end_time_date, end_time_hour), mean_realvalue=mean(ifm_analysis_sensordata_for_datathon_final.realvalue,na.rm = TRUE))
        head(Agg_day_hour_RBG1_drive_gear_a_max)

        # unite date & hour
        Agg_day_hour_RBG1_drive_gear_a_max1 = unite(Agg_day_hour_RBG1_drive_gear_a_max, end_time_date_hour, end_time_date, end_time_hour, sep = ',')
        head(Agg_day_hour_RBG1_drive_gear_a_max1)

        # outlier df
        outs = boxplot.stats(Agg_day_hour_RBG1_drive_gear_a_max1$mean_realvalue)$out
        outlier_Agg_day_hour_RBG1_drive_gear_a_max1 = Agg_day_hour_RBG1_drive_gear_a_max1[Agg_day_hour_RBG1_drive_gear_a_max1$mean_realvalue %in% outs,]
        write.csv(outlier_Agg_day_hour_RBG1_drive_gear_a_max1, paste(folder, "/out_", names(all_csv)[i], ".csv", sep=""))

        #outlier number
        outlier_num = length(outs)

        # df without outliers
        Agg_day_hour_withoutOutlier_RBG1_drive_gear_a_max1 = Agg_day_hour_RBG1_drive_gear_a_max1[!(Agg_day_hour_RBG1_drive_gear_a_max1$mean_realvalue %in% outs),]
        write.csv(Agg_day_hour_withoutOutlier_RBG1_drive_gear_a_max1, paste(folder, "/Without_out_", names(all_csv)[i], ".csv", sep=""))

        newRow = data.frame(sensor_name= names(all_csv)[i], 
                            outlier_number=outlier_num )
        boundary = rbind(boundary, newRow)

    }
    write.csv(boundary, paste(folder, "/NumberofOutlier.csv", sep=""))
}